In [1]:
import sys
!{sys.executable} -m pip install python-dotenv==0.19.2 PyMySQL==1.0.2 numpy==1.20.2 pandas==1.2.4

In [2]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import pymysql

pd.set_option('display.max_columns', 40)

USE_DATABASE = False
SQL_TABLENAME_BOOTSTRAP = "results_rr"
SQL_TABLENAME_GENERALIZATION = "generalization"

CSV_CROSSPARE_BOOTSTRAP = "data/database_metrics_vs_costsaving_bt_exp.csv"
CSV_CROSSPARE_GENERALIZATION = "data/database_metrics_vs_costsaving_real.csv"

CSV_OUTPUT_BOOTSTRAP = "data/metrics_vs_costsaving_bootstrap_experiment.csv"
CSV_OUTPUT_GENERALIZATION = "data/metrics_vs_costsaving_realistic_settings.csv"

if USE_DATABASE:
    load_dotenv("db_credentials.env")

### Functions

In [3]:
def convert_dataframe(mysql_df : pd.DataFrame):
    """Converts a dataframe from CrossPare's MySQL database format to make it fitting for the analysis.
    
    The conversion is done in multiple steps: First the columns getting renamed.
    Afterwards the missing columns are computed. And finally the relevant columns are selected.
    
    Args:
        mysql_df: original dataframe
        
    Returns:
        pd.Dataframe: converted dataframe
    """
    df = mysql_df.copy(deep=True)

    # rename columns
    df.rename(columns = {
                     'fscore':'f_measure',
                     'gscore':'g_measure',
                     'aucAlberg':'auc_alberg',
                     'aucRoI':'auc_roi',
                     'nofi80':'nofc80',
                     'biasTrainingOriginal':'bias_train',
                     'biasTraining':'bias_train_processed',
                     'biasTest':'bias_test',
                     'prop1Defect':'prop1_defect',
                     'prop1Clean':'prop1_clean',
                     'trainsizeOriginal':'num_train',
                     'trainsize':'num_train_processed',
                     'testsize':'num_test'
                        }, inplace = True)

    # treat edge cases of existing variables
    df.loc[ (df['tp']==0) & (df['fn']==0), 'recall' ] = 1 # The model predicted "all" of the none defective instances (1)
    df.loc[ (df['tp']==0) & (df['fp']==0), 'precision'] = 1 # if nothing is predicted as defective, the precision is 1.
    df['f_measure'] = np.where( ( df['recall'] + df['precision'] )!=0,
                                2 * ( df['recall'] * df['precision'] ) / ( df['recall'] + df['precision'] ),
                                0)
    df['g_measure'] = np.where( ( df['recall'] + ( 1 - df['fpr'] ) )!=0,
                                2 * ( df['recall'] * ( 1 - df['fpr'] ) ) / ( df['recall'] + ( 1 - df['fpr'] ) ),
                                0)
    df['balance'] = 1 - ( ( (1-df['recall'])**2 + df['fpr']**2 )**(1/2) / 2**(1/2) )
    df['mcc'] = np.where( ( (df['tp']+df['fp'])*(df['tp']+df['fn'])*(df['tn']+df['fp'])*(df['tn']+df['fn']) )!=0,
                          df['mcc'],
                          0)
    df.loc[ (df['tp']==0) & (df['fn']==0), 'auc' ] = np.nan
    df.loc[ (df['tp']==0) & (df['fn']==0), 'auc_alberg' ] = np.nan
    df.loc[ (df['tp']==0) & (df['fn']==0), 'auc_roi' ] = np.nan
    df['nofc80'] = np.where( (df['tp']+df['fn'])!=0,
                             df['nofc80'],
                             np.nan)
    df['prop1_defect'] = np.where( (df['tp']+df['fn'])!=0,
                                   df['prop1_defect'],
                                   np.nan)

    # calculate missing variables
    df["fpr"] = df["fp"] / (df["tn"] + df["fp"])
    df["accuracy"] = (df["tp"] + df["tn"]) / df["num_test"]
    df["error_type1"] = np.where( ( df["tp"] + df["fn"] )!=0,
                                  ( df["fp"] / (df["tp"] + df["fn"]) ),
                                  np.inf)
    df["error_type2"] = df["fn"] / (df["tn"] + df["fp"])
    df["consistency"] = np.where( ( ( df["tp"] + df["fn"] ) * ( df["tn"] + df["fp"] ) )!=0,
                                  ( ( df["tp"] * df["num_test"] ) - ( df["tp"] + df["fn"] )**2) / ( ( df["tp"] + df["fn"] ) * ( df["tn"] + df["fp"] ) ),
                                  np.nan)
    df["ratio_bias"] = df["bias_test"] / df["bias_train"]
    df["ratio_bias_processed"] = np.where(df["bias_train_processed"]!=0,
                                          df["bias_test"] / df["bias_train_processed"],
                                          np.nan)

    # calculate diff value
    df["diff"] = df["upperSizeNtoM"] - df["lowerSizeNtoM"]
    df['unregular_lower_bound'] = "regular"
    df['unregular_upper_bound'] = "regular"
    df.loc[(df['tp'] ==0) &  (df['fp'] !=0) , 'unregular_lower_bound'] = '+inf'
    df.loc[(df['tp'] ==0) &  (df['fp'] ==0) , 'unregular_lower_bound'] = 'NaN'
    df.loc[(df['fn'] ==0) &  (df['tn'] !=0) , 'unregular_upper_bound'] = '+inf'
    df.loc[(df['fn'] ==0) &  (df['tn'] ==0) , 'unregular_upper_bound'] = 'NaN'
    df.loc[(df['unregular_lower_bound'] =='NaN') |  (df['unregular_upper_bound'] =='Nan') , 'diff'] = np.nan
    df.loc[(df['unregular_lower_bound'] =='+inf') &  (df['unregular_upper_bound'] =='+inf') , 'diff'] = np.nan
    df.loc[(df['unregular_lower_bound'] =='+inf') &  (df['unregular_upper_bound'] =='regular') , 'diff'] = -np.inf
    df.loc[(df['unregular_lower_bound'] =='regular') &  (df['unregular_upper_bound'] =='+inf') , 'diff'] = np.inf

    # get cost saving potential classes
    conditions = [
        (df['diff'] <= 0) | (df['diff'] == -np.inf) | (df['diff'].isnull()),
        (df['diff'] > 0) & (df['diff'] <= 10),
        (df['diff'] > 10) & (df['diff'] <= 100),
        (df['diff'] > 100) & (df['diff'] <= 1000),
        (df['diff'] > 1000) & (df['diff'] <= 10000),
        (df['diff'] > 10000 | (df['diff'] == np.inf))
        ]
    # create a list of the values we want to assign for each condition
    potentials = ['none', 'negligible', 'small', 'medium', 'large', 'extra_large']
    # create a new column and use np.select to assign values to it using our lists as arguments
    df['potential'] = np.select(conditions, potentials)

    # pick relevant columns
    independent_variables = ["recall", "precision", "fpr", "f_measure", "g_measure", "balance", "accuracy",
                         "error", "error_type1", "error_type2", "mcc", "consistency", "auc", "auc_alberg", "auc_roi",
                         "necm10", "necm25", "cost", "nofb20", "nofc80"]
    confounding_variables = ["bias_train", "bias_train_processed", "bias_test", "ratio_bias", "ratio_bias_processed",
                             "prop1_defect", "prop1_clean", "num_train", "num_train_processed", "num_test"]
    column_list = ["configuration"] + ["release"] + independent_variables + confounding_variables + ["diff"] + ["potential"]
    df = df[column_list]
    return df

def few_descriptive_stats(df):
    """Returns a dataframe containing descriptive statistics for detecting invalid values."""
    desc_stats = pd.DataFrame(columns=df.columns)
    desc_stats = desc_stats.append((df.mean(axis=0).rename("mean")))
    desc_stats = desc_stats.append((df.median(axis=0).rename("median")))
    desc_stats = desc_stats.append(df.min(axis=0, numeric_only=True).rename("min"))
    desc_stats = desc_stats.append(df.max(axis=0, numeric_only=True).rename("max"))
    return desc_stats

def nan_and_inf_evaluation(df):
    """Prints information about the occurrences of NaN values in the dataset"""
    print(f"Overall number of NaN:               {df.isna().sum().sum()}")
    print(f"Overall number of NaN (no 'diff'):   {df.drop('diff', axis=1).isna().sum().sum()}")
    print(f"Number of rows with NaN:             {df[df.isna().any(axis=1)].shape[0]}")
    print(f"Number of rows with NaN (no 'diff'): {df[df.drop('diff', axis=1).isna().any(axis=1)].shape[0]}")
    print("\nNaNs per variable:")
    print(df.isna().sum().loc[df.isna().sum()>0] )
    print("\ninfs per variable:")
    inf_array = np.isinf(df.drop(["configuration","release","potential"], axis=1)).sum()
    print(inf_array[inf_array>0])

### Connect to database

In [4]:
if USE_DATABASE:
    db_host = os.getenv('DB_HOST')
    db_port = int(os.getenv('DB_PORT'))
    db_user = os.getenv('DB_USER')
    db_pass = os.getenv('DB_PASS')
    db_name = os.getenv('DB_NAME')
    dbcon = pymysql.connect(host=db_host, user=db_user, password=db_pass, database=db_name, port=db_port)

# get bootstrap experiment data
### Query bootstrap experiment dataframe

In [5]:
if USE_DATABASE:
    db_table = SQL_TABLENAME_BOOTSTRAP
    bt_df = pd.read_sql_query(f"SELECT * FROM {db_table} WHERE configurationName LIKE '%NoSmote%' OR configurationName LIKE '%Smotuned%'", dbcon)

    bt_df.to_csv(CSV_CROSSPARE_BOOTSTRAP, index=False)
else:
    bt_df = pd.read_csv(CSV_CROSSPARE_BOOTSTRAP,index_col=False)

print(f"shape of dataframe: {bt_df.shape}")

release_col = bt_df["productName"].str.replace("_aggregated.csv", "", regex=True)
bt_df.insert(0, 'release', release_col)

config_col = bt_df['configurationName'].apply(lambda x: 'Smotuned' if "Smotuned" in x else 'NoSmote')
bt_df.insert(0, 'configuration', config_col)

few_descriptive_stats(bt_df)

shape of dataframe: (53000, 115)


,configuration,release,idresults,configurationName,productName,classifier,testsize,trainsize,trainsizeOriginal,biasTest,biasTraining,biasTrainingOriginal,prop1Defect,prop1Clean,error,recall,precision,fscore,gscore,mcc,...,lowerConstNtoMImp40,upperConstNtoMImp40,lowerSize1to1Imp40,upperSize1to1Imp40,lowerSize1toMImp40,upperSize1toMImp40,lowerSizeNtoMImp40,upperSizeNtoMImp40,lowerConst1to1Imp50,upperConst1to1Imp50,lowerConst1toMImp50,upperConst1toMImp50,lowerConstNtoMImp50,upperConstNtoMImp50,lowerSize1to1Imp50,upperSize1to1Imp50,lowerSize1toMImp50,upperSize1toMImp50,lowerSizeNtoMImp50,upperSizeNtoMImp50
mean,NaN,NaN,46019.841868,NaN,NaN,NaN,186.097717,560.944302,506.490566,0.065939,0.150173,0.065685,152.347081,15.084337,0.071236,0.171032,0.066191,-0.034434,0.239728,-0.273583,...,0.329204,94.173707,1614.561656,4994.255011,1220.851624,4382.491603,188.967099,7.093923e+03,2.284447,61.427844,1.702248,54.499107,0.361874,4.817570e+02,1937.563029,5993.108213,1465.110990,5258.992124,194.965369,2.495201e+04
median,NaN,NaN,52424.000000,NaN,NaN,NaN,153.000000,462.000000,425.000000,0.054348,0.108252,0.054417,83.400000,10.806122,0.059406,0.090909,0.250000,0.142857,0.166604,0.129023,...,0.211864,37.666667,439.226190,3325.555556,318.888889,2850.458333,82.994845,3.492925e+03,2.000000,42.857143,1.500000,37.000000,0.214286,4.586667e+01,527.071429,3990.666667,382.666667,3420.550000,83.755000,4.249163e+03
min,NaN,NaN,1.000000,NaN,NaN,NaN,28.000000,100.000000,100.000000,0.002481,0.002347,0.002347,5.600000,1.819095,0.000000,0.000000,-1.000000,-1.000000,0.000000,-2.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00
max,NaN,NaN,79500.000000,NaN,NaN,NaN,667.000000,1940.000000,1708.000000,0.382353,0.692641,0.333333,3756.000000,92.700000,0.400000,1.000000,1.000000,1.000000,1.000000,1.000000,...,77.160494,338164.776026,65188.333333,120001.666667,65188.333333,120001.666667,12297.222222,1.326499e+07,42.000000,804.000000,42.000000,804.000000,192.000000,5.210112e+06,78226.000000,144002.000000,78226.000000,144002.000000,17708.000000,2.043740e+08


### Convert bootstrap dataframe

In [6]:
bt_df_converted = convert_dataframe(bt_df)
few_descriptive_stats(bt_df_converted)

,configuration,release,recall,precision,fpr,f_measure,g_measure,balance,accuracy,error,error_type1,error_type2,mcc,consistency,auc,auc_alberg,auc_roi,necm10,necm25,cost,nofb20,nofc80,bias_train,bias_train_processed,bias_test,ratio_bias,ratio_bias_processed,prop1_defect,prop1_clean,num_train,num_train_processed,num_test,diff,potential
mean,NaN,NaN,0.171032,0.521851,0.022190,0.193396,0.239728,0.412477,0.928764,0.071236,0.326751,0.056615,0.182078,0.114417,0.790624,0.013403,0.575275,0.532306,1.300757,3579.802868,2.763623,76.475075,0.065685,0.150173,0.065939,1.142955,0.726042,152.347081,15.084337,506.490566,560.944302,186.097717,NaN,NaN
median,NaN,NaN,0.090909,0.500000,0.010989,0.142857,0.166604,0.357168,0.940594,0.059406,0.190476,0.047120,0.129023,0.015152,0.807238,0.006870,0.572759,0.461538,1.126795,974.000000,2.000000,54.000000,0.054417,0.108252,0.054348,1.004008,0.496633,83.400000,10.806122,425.000000,462.000000,153.000000,990.309524,NaN
min,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.270018,0.600000,0.000000,0.000000,0.000000,-0.173902,-0.466667,0.055556,0.000000,0.012178,0.000000,0.000000,0.000000,0.000000,1.000000,0.002347,0.002347,0.002481,0.066808,0.023711,5.600000,1.819095,100.000000,100.000000,28.000000,-inf,NaN
max,NaN,NaN,1.000000,1.000000,0.404762,1.000000,1.000000,1.000000,1.000000,0.400000,12.000000,0.476190,1.000000,1.000000,1.000000,0.185233,1.000000,3.181818,7.954545,56593.000000,35.000000,634.000000,0.333333,0.692641,0.382353,14.300000,14.300000,3756.000000,92.700000,1708.000000,1940.000000,667.000000,inf,NaN


### Check for the completeness of bootstrap experiment data
#### expected values:
- number of releases = 265
- instances per release = 200 (2*100)
- total number of instances = 53000 (2*26500)

In [7]:
num_releases = bt_df_converted["release"].nunique()
instances_per_release = bt_df_converted["num_test"].groupby(bt_df_converted["release"]).count().unique().tolist()
print(f"Number of releases:        {num_releases}")
print(f"Instances per release:     {instances_per_release}")
print(f"Total number of instances: {bt_df_converted.shape[0]}\n")
nan_and_inf_evaluation(bt_df_converted)

Number of releases:        265
Instances per release:     [200]
Total number of instances: 53000

Overall number of NaN:               12075
Overall number of NaN (no 'diff'):   0
Number of rows with NaN:             12075
Number of rows with NaN (no 'diff'): 0

NaNs per variable:
diff    12075
dtype: int64

infs per variable:
diff    12104
dtype: int64


### Save converted bootstrap experiment dataframe to csv file

In [8]:
bt_df_converted.to_csv(CSV_OUTPUT_BOOTSTRAP)

# get generalization experiment data
### Query generalization experiment dataframe

In [9]:
if USE_DATABASE:
    db_table = SQL_TABLENAME_GENERALIZATION
    real_df = pd.read_sql_query(f"SELECT * FROM {db_table} WHERE configurationName NOT LIKE '%NoSmote%' AND configurationName NOT LIKE '%Smotuned%'", dbcon)
    real_df.to_csv(CSV_CROSSPARE_GENERALIZATION, index=False)
else:
    real_df = pd.read_csv(CSV_CROSSPARE_GENERALIZATION,index_col=False)

print(f"shape of dataframe: {real_df.shape}")

release_col = real_df["productName"].str.replace("_aggregated.csv", "", regex=True)
real_df.insert(0, 'release', release_col)

config_col = real_df['configurationName'].str.replace("MYNBOU_", "", regex=True)
real_df.insert(0, 'configuration', config_col)

few_descriptive_stats(real_df)

shape of dataframe: (1698, 115)


,configuration,release,idresults,configurationName,productName,classifier,testsize,trainsize,trainsizeOriginal,biasTest,biasTraining,biasTrainingOriginal,prop1Defect,prop1Clean,error,recall,precision,fscore,gscore,mcc,...,lowerConstNtoMImp40,upperConstNtoMImp40,lowerSize1to1Imp40,upperSize1to1Imp40,lowerSize1toMImp40,upperSize1toMImp40,lowerSizeNtoMImp40,upperSizeNtoMImp40,lowerConst1to1Imp50,upperConst1to1Imp50,lowerConst1toMImp50,upperConst1toMImp50,lowerConstNtoMImp50,upperConstNtoMImp50,lowerSize1to1Imp50,upperSize1to1Imp50,lowerSize1toMImp50,upperSize1toMImp50,lowerSizeNtoMImp50,upperSizeNtoMImp50
mean,NaN,NaN,78909.5,NaN,NaN,NaN,486.586572,25254.421673,25262.964664,0.064404,0.047983,0.048029,92.169938,14.794172,0.257466,0.459338,0.172051,0.152568,0.404748,0.080306,...,24.941260,2.361026e+04,5003.268281,4867.556762,4149.515320,4447.226029,5888.258576,1.340923e+06,23.318783,91.158914,20.267218,84.051431,32.157967,1.867605e+06,6003.939369,5841.102861,4979.435817,5336.695616,7433.763590,1.060386e+08
median,NaN,NaN,78909.5,NaN,NaN,NaN,403.500000,6557.000000,6557.000000,0.052397,0.052895,0.052907,53.043810,10.487179,0.180602,0.500000,0.133631,0.186141,0.504011,0.155656,...,10.488240,4.919852e+01,3076.303419,2464.542824,2336.944444,2254.625000,3381.834638,3.441435e+03,12.000000,42.119298,9.376518,36.829167,12.695798,6.090598e+01,3691.564103,2957.451389,2804.333333,2705.550000,4109.079365,4.259188e+03
min,NaN,NaN,78061.0,NaN,NaN,NaN,29.000000,81.000000,100.000000,0.000000,0.000000,0.005025,-1.000000,2.542857,0.001441,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00
max,NaN,NaN,79758.0,NaN,NaN,NaN,1708.000000,128751.000000,128751.000000,0.285714,0.126394,0.126394,1184.000000,50.171569,0.990411,1.000000,1.000000,0.727273,0.975207,0.714851,...,1805.555556,2.089321e+07,90085.000000,122528.333333,90085.000000,122528.333333,262345.679012,1.218560e+09,786.000000,1612.000000,786.000000,1612.000000,3744.000000,1.660944e+09,108102.000000,147034.000000,108102.000000,147034.000000,544000.000000,9.687165e+10


### Convert bootstrap dataframe

In [10]:
real_df_converted = convert_dataframe(real_df)
few_descriptive_stats(real_df_converted)

,configuration,release,recall,precision,fpr,f_measure,g_measure,balance,accuracy,error,error_type1,error_type2,mcc,consistency,auc,auc_alberg,auc_roi,necm10,necm25,cost,nofb20,nofc80,bias_train,bias_train_processed,bias_test,ratio_bias,ratio_bias_processed,prop1_defect,prop1_clean,num_train,num_train_processed,num_test,diff,potential
mean,NaN,NaN,0.544143,0.180296,0.268267,0.199093,0.479825,0.548689,0.715989,0.257466,inf,0.034347,0.159812,0.537867,0.713829,0.006730,0.526263,0.534241,0.995533,32011.067138,5.942874,257.845633,0.048029,0.047983,0.064404,1.726356,1.726287,96.295544,14.794172,25262.964664,25254.421673,486.586572,NaN,NaN
median,NaN,NaN,0.538462,0.135129,0.156124,0.186141,0.528660,0.549462,0.819398,0.180602,2.857143,0.021529,0.155656,0.483689,0.741370,0.001469,0.528006,0.488062,0.782382,15260.000000,4.000000,180.000000,0.052907,0.052895,0.052397,1.243741,1.237294,55.652778,10.487179,6557.000000,6557.000000,403.500000,78.004924,NaN
min,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.134187,0.000000,0.001441,0.000000,0.000000,-0.365541,-0.247191,0.017964,0.000000,0.095069,0.001441,0.001441,0.000000,0.000000,1.000000,0.005025,0.000000,0.000000,0.000000,0.000000,5.678571,2.542857,100.000000,81.000000,29.000000,-inf,NaN
max,NaN,NaN,1.000000,1.000000,1.000000,0.727273,0.999279,0.998981,0.998559,0.990411,inf,0.254545,0.714851,4.235294,1.000000,0.095691,1.000000,2.004505,4.977477,187780.000000,83.000000,1630.000000,0.126394,0.126394,0.285714,17.424007,18.949020,1184.000000,50.171569,128751.000000,128751.000000,1708.000000,inf,NaN


### Check for the completeness of generalization experiment data
#### expected values:
- number of releases = 398
- instances per release = 0-6
- maximal total number of instances = 6*398 = 2388

In [11]:
num_releases = real_df_converted["release"].nunique()
instances_per_release = real_df_converted["num_test"].groupby(real_df_converted["release"]).count().unique().tolist()
print(f"Number of releases:        {num_releases}")
print(f"Instances per release:     {instances_per_release}")
print(f"Total number of instances: {real_df_converted.shape[0]}\n")
nan_and_inf_evaluation(real_df_converted)

Number of releases:        337
Instances per release:     [3, 6]
Total number of instances: 1698

Overall number of NaN:               512
Overall number of NaN (no 'diff'):   434
Number of rows with NaN:             79
Number of rows with NaN (no 'diff'): 74

NaNs per variable:
consistency             72
auc                     72
auc_alberg              72
auc_roi                 72
nofc80                  72
ratio_bias_processed     2
prop1_defect            72
diff                    78
dtype: int64

infs per variable:
error_type1     72
diff           236
dtype: int64


### Save converted generalization experiment dataframe to csv file


In [12]:
real_df_converted.to_csv(CSV_OUTPUT_GENERALIZATION)